In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
import os
import statsmodels.api as sm
#from graphviz import Source
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
#from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import mapi
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [2]:
# new imports
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV

In [3]:
#Load Raw Data
file_path = 'C:\\Users\\rafma\\Python\\46889 Capstone\\GenericWellData.xlsx'
df = pd.read_excel(file_path)
index = df['Well Index']

In [4]:
#Remove Date fields from training data set
#Dropped also 'MOnthsProductionCount'
#Reponse variable limited to only first 3 months production output oil
y = df['CumOil_BBL']
x = df.drop(['FractureStages', 'AvgStageSpacing_FT', 'Well Index', 'InitialProductionDate', 'DrillingStartDate','InitialProductionYear', 'DrillingCompletionDate', 'YearOfDrilling', 'InitialProductionMonth', 'ProductionMonthsCount', 'First36MonthWater_BBL', 'First3MonthGas_MCF', 'First3MonthGas_MCFPer1000FT', 'First3MonthProd_MCFE', 'First3MonthProd_MCFEPer1000FT', 'First3MonthOil_BBL', 'First3MonthOil_BBLPer1000FT', 'First3MonthWater_BBL', 'First6MonthProd_BOE', 'First6MonthProd_BOEPer1000FT', 'First6MonthGas_MCF', 'First6MonthGas_MCFPer1000FT', 'First6MonthProd_MCFE', 'First6MonthProd_MCFEPer1000FT', 'First6MonthOil_BBL', 'First6MonthOil_BBLPer1000FT', 'First6MonthWater_BBL', 'First9MonthProd_BOE', 'First9MonthProd_BOEPer1000FT', 'First9MonthGas_MCF', 'First9MonthGas_MCFPer1000FT', 'First9MonthProd_MCFE', 'First9MonthProd_MCFEPer1000FT', 'First9MonthOil_BBL', 'First9MonthOil_BBLPer1000FT', 'First9MonthWater_BBL', 'First12MonthProd_BOE', 'First12MonthProd_BOEPer1000FT', 'First12MonthGas_MCF', 'First12MonthGas_MCFPer1000FT', 'First12MonthProd_MCFE', 'First12MonthProd_MCFEPer1000FT', 'First12MonthOil_BBL', 'First12MonthOil_BBLPer1000FT', 'First12MonthWater_BBL', 'First36MonthProd_BOE', 'First36MonthProd_BOEPer1000FT', 'First36MonthGas_MCFPer1000FT', 'First36MonthProd_MCFE', 'First36MonthProd_MCFEPer1000FT', 'First36MonthOil_BBLPer1000FT', 'First36MonthWaterProductionBBLPer1000Ft', 'PeakProd_BOE', 'PeakProd_BOEPer1000FT', 'PeakGas_MCF', 'PeakGas_MCFPer1000FT', 'PeakProd_MCFE', 'PeakProd_MCFEPer1000FT', 'PeakOil_BBL', 'PeakOil_BBLPer1000FT', 'PeakWater_BBL', 'CumProd_BOE', 'CumProd_BOEPer1000FT', 'CumGas_MCF', 'CumGas_MCFPer1000FT', 'CumProd_MCFE', 'CumProd_MCFEPer1000FT', 'CumOil_BBL', 'CumOil_BBLPer1000FT', 'CumWater_BBL', 'First3MonthProd_BOEPer1000FT'], axis = 1)
#'Well Index'

In [5]:
x = pd.get_dummies(x, columns=['OilTest_Method'])
for col in x.columns:
    if x[col].dtype == 'bool':
        x[col] = x[col].astype(int)

In [6]:
#Convert float data type to numeric and remove comma separators of numbers
for col in x.columns:
    if x[col].dtype == 'object':  # Check if the column contains string values
        # Remove commas and convert to numeric, setting errors to 'coerce' to handle non-numeric strings
        x[col] = pd.to_numeric(x[col].str.replace(',', ''), errors='coerce')

In [7]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16707 entries, 0 to 16706
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   TrueVerticalDepth_FT        16705 non-null  float64
 1   MeasuredDepth_FT            16660 non-null  float64
 2   UpperPerforation_FT         16573 non-null  float64
 3   LowerPerforation_FT         16573 non-null  float64
 4   PerforationInterval_FT      16659 non-null  float64
 5   LateralLength_FT            16705 non-null  float64
 6   ProppantLoad_LBSPerGAL      15188 non-null  float64
 7   ProppantIntensity_LBSPerFT  15324 non-null  float64
 8   TotalProppant_LBS           15368 non-null  float64
 9   TotalWaterPumped_GAL        15682 non-null  float64
 10  WaterIntensity_GALPerFT     15662 non-null  float64
 11  TotalFluidPumped_BBL        15662 non-null  float64
 12  FluidIntensity_BBLPerFT     15642 non-null  float64
 13  AcidVolume_BBL              134

In [8]:
#Train Test Split for XGBOOST KNN Optimal k determination
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=99)

In [11]:
# Dictionary to store the scores for different k values
scores = {}

# Iterate over multiple k values
for k in range(1, 15):  # Example: trying k values from 1 to 5
    #Create kNN Imputer
    imputer = KNNImputer(n_neighbors=k)
    
    # Fit the imputer and transform the training data
    x_train_imputed = imputer.fit_transform(x_train)
    x_test_imputed = imputer.transform(x_test)
    
    # Fit a simple model (RF OR XGBOOST) to the imputed data
    #model = RandomForestRegressor(random_state=99) if RF model desired for optimal k determination
    model = XGBRegressor(learning_rate=0.075, n_estimators= 400, max_depth = 7, random_state=99, alpha = 0.5)
    model.fit(x_train_imputed, y_train)
    
    # Predict on the test set and evaluate the performance
    y_pred = model.predict(x_test_imputed)
    #accuracy = accuracy_score(y_test, y_pred)
    
    #Evaluate the performance using R-squared or or other performance metric
    r2 = r2_score(y_test, y_pred)
    #mae = mean_absolute_error(y_test, y_pred)
    #mse = mean_squared_error(y_test, y_pred)
   # rmse = np.sqrt(mse)
    
    # Store the score
    scores[k] = r2
    #scores[k] = accuracy
    
# Find the best k value
best_k = max(scores, key=scores.get)
best_score = scores[best_k]

print(f"Best k: {best_k} with R^2: {best_score}")
print(scores)

Best k: 7 with accuracy: 0.29544534004467893
{1: 0.2677167482611862, 2: 0.2806797344155827, 3: 0.28054629367483874, 4: 0.2895817402455748, 5: 0.280867496665981, 6: 0.28018913185062855, 7: 0.29544534004467893, 8: 0.2847699056619184, 9: 0.28614912973351647, 10: 0.27377644805617574, 11: 0.28544787502021074, 12: 0.28667449512613186, 13: 0.28031968571168886, 14: 0.28736694304752886}


In [12]:
#Imputation for k= 7
imputer = IterativeImputer(estimator=KNeighborsRegressor(n_neighbors=7))

In [13]:
#Runs the KNN iterative imputer to estimate missing values for the :
imputed_data = imputer.fit_transform(x)
imputed_x = pd.DataFrame(imputed_data, columns=x.columns)

C:\Users\rafma\anaconda3\Lib\site-packages\sklearn\impute\_iterative.py:796: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [14]:
imputed_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16707 entries, 0 to 16706
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   TrueVerticalDepth_FT        16707 non-null  float64
 1   MeasuredDepth_FT            16707 non-null  float64
 2   UpperPerforation_FT         16707 non-null  float64
 3   LowerPerforation_FT         16707 non-null  float64
 4   PerforationInterval_FT      16707 non-null  float64
 5   LateralLength_FT            16707 non-null  float64
 6   ProppantLoad_LBSPerGAL      16707 non-null  float64
 7   ProppantIntensity_LBSPerFT  16707 non-null  float64
 8   TotalProppant_LBS           16707 non-null  float64
 9   TotalWaterPumped_GAL        16707 non-null  float64
 10  WaterIntensity_GALPerFT     16707 non-null  float64
 11  TotalFluidPumped_BBL        16707 non-null  float64
 12  FluidIntensity_BBLPerFT     16707 non-null  float64
 13  AcidVolume_BBL              167